<a href="https://colab.research.google.com/github/Hummna/Core-Human-Facial-Emotions-Detection./blob/main/DeepLearning_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Change directory to the desired location in Google Drive
import os
os.chdir('/content/gdrive/MyDrive')

# Import required libraries
import numpy as np
import pandas as pd
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
# Read the training dataset
df_train = pd.read_csv('/content/gdrive/MyDrive/DL_Dataset/Train/train.csv')

# Display the first few rows of the dataset
df_train=df_train.convert_dtypes()
df_train.head()

,img,classname
0,img_292,0
1,img_316,0
2,img_344,0
3,img_375,0
4,img_498,0


In [ ]:
# Create one-hot encoded labels
targets_series = pd.Series(df_train['classname'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [ ]:
# Set image size
im_size1 = 128
im_size2 = 128
BatchSize = 32

# Prepare the training data
x_train = []
y_train = []
x_test = []


i = 0
for f, breed in tqdm(df_train.values):
   # Check if the image exists
    if type(cv2.imread('/content/gdrive/MyDrive/DL_Dataset/Train/train_dataset/img_232.jpg'.format(f)))==type(None):
        continue
    else:
        img = cv2.imread('/content/gdrive/MyDrive/DL_Dataset/Train/train_dataset/img_232.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size1, im_size2)))
        y_train.append(label)
        #print(x_train[i])
        #print(label[i])
        i += 1

# Save the training data
np.save('/content/gdrive/MyDrive/x_train',x_train)
np.save('/content/gdrive/MyDrive/y_train',y_train)
print('Done')

<ipython-input-23-a5ea09c15746>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for f, breed in tqdm(df_train.values):


  0%|          | 0/58 [00:00<?, ?it/s]

Done


In [ ]:
# Load the training data
x_train = np.load('/content/gdrive/MyDrive/x_train.npy')
y_train = np.load('/content/gdrive/MyDrive/y_train.npy')
np.expand_dims(x_train, axis=-1)


# Display the shapes of the training data
print(x_train.shape)
print(y_train.shape)

(58, 128, 128, 3)
(58, 10)


In [ ]:
# Preprocess the training data
y_train_raw = np.array(y_train, np.uint8)
#x_train_raw = np.array(X_train, np.float32)
x_train_raw = np.array(y_train, np.float32) / 255.

y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32)/255


print(x_train_raw.shape)
print(y_train_raw.shape)

(58, 128, 128, 3)
(58, 10)


In [ ]:
# Split the training data into train and validation sets
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.4, train_size=0.6, random_state=1)


# Display the shapes of the split data
print(X_train.shape)
print(Y_train.shape)

# Get the number of classes
num_class = y_train_raw.shape[1]


print(num_class)

(34, 128, 128, 3)
(34, 10)
10


In [ ]:
#Below is the website you will get info about pre train models.
#https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16

In [ ]:
# Import the VGG16 model
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights = 'imagenet', include_top=False, input_shape=(im_size1, im_size2, 3))

In [ ]:
# Add new layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)

#activation = 'sigmoid' in case of binary classification
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False



# Unfreeze the last 4 layers for training

# for layer in model.layers[-4:]:
#     layer.trainable = True


# Compile the model
#loss = binary_crossentropy --> in case of binary classification

#list of optimizers that can be used:
#SGD (Stochastic Gradient Descent)
#RMSprop
#Adagrad
#Adadelta
#Adamax
#Nadam
#AdamW
#AMSGrad
#Adafactor
#Ftrl
#TFOptimizer



model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Define callbacks
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]

# Display the model summary
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0   

In [ ]:
# Train the model
history = model.fit(X_train, Y_train, epochs=1, batch_size= BatchSize , validation_data=(X_valid, Y_valid), verbose=1)

2/2 [==============================] - 33s 6s/step - loss: 3.3223 - accuracy: 0.0882 - val_loss: 3.3024 - val_accuracy: 0.1667


In [ ]:
# Save the trained model
model.save('/content/gdrive/MyDrive/Afaq_presentation/model.h5')
print('Model saved.')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved.


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the test data
df_test = pd.read_csv('/content/gdrive/MyDrive/DL_Dataset/Test/test.csv')
df_test = df_test.convert_dtypes()
x_test = []
y_test = []
test_filenames = []

In [ ]:
for f, classname in tqdm(df_test[['img', 'classname']].values):
    if type(cv2.imread('/content/gdrive/MyDrive/DL_Dataset/Test/test_dataset/img_108.jpg'.format(f))) == type(None):
        continue
    else:
        img = cv2.imread('/content/gdrive/MyDrive/DL_Dataset/Test/test_dataset/img_108.jpg'.format(f))
        x_test.append(cv2.resize(img, (im_size1, im_size2)))
        y_test.append(classname)
        test_filenames.append(f)


<ipython-input-33-7d9401159caf>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for f, classname in tqdm(df_test[['img', 'classname']].values):


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
# Convert lists to numpy arrays
x_test = np.array(x_test)
y_test = np.array(y_test)

# Normalize the input images
x_test = x_test.astype('float32') / 255.

In [ ]:
# Load the trained model
model = keras.models.load_model('/content/gdrive/MyDrive/Afaq_presentation/model.h5')

# Make predictions on the test data
predictions = model.predict(x_test)

# Convert predictions to class labels
predicted_labels = [one_hot.columns[np.argmax(pred)] for pred in predictions]


1/1 [==============================] - 9s 9s/step


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print('Accuracy on testing dataset:', accuracy)



Accuracy on testing dataset: 0.1


In [ ]:
# Create a confusion matrix
cm = confusion_matrix(y_test, predicted_labels)
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0 0]]


In [ ]:
# Generate classification report
report = classification_report(y_test, predicted_labels, zero_division=0)
print('Classification Report:\n', report)

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         3
           6       0.10      1.00      0.18         3
           7       0.00      0.00      0.00         3
           8       0.00      0.00      0.00         3
           9       0.00      0.00      0.00         3

    accuracy                           0.10        30
   macro avg       0.01      0.10      0.02        30
weighted avg       0.01      0.10      0.02        30

